#To install git on aws ec2 instance:
1. git --version # to get the version info
2. sudo yum remove git -y
3. sudo yum install git -y
4. git login

#To install jenkins on aws ec2 instance linux:
1. How to download and install jenkins on aws ec2 linux
2. How to access Jenkins from browser
3. Hot to start and stop Jenkins
4. How to start jenkins on different port
5. How to uninstall jenkins

#To install Jenkins:
1. sudo yum update
2. java -version #in case if it is not java 8, 
3. to check and select one out of multiple java versions available: sudo /usr/sbin/alternatives --config java
4. sudo yum install java-1.8.0 #if java is not installed
Download latest jenkins code package
5. sudo wget -O /etc/yum.repos.d/jenkins.repo http://pkg.jenkins-ci.org/redhat/jenkins.repo
Import a key file from Jenkins-CI to enalbe installation from the package
6. sudo rpm --import http://pkg.jenkins-ci.org/redhat/jenkins-ci.org.key
to install jenkins now
7. sudo yum install jenkins
to start jenkins
8. sudo service jenkins start
to access jenkins server using the public dns of your ec2 on port 8080
9. http://ec2-public-dns:8080 example: http://3.89.79.74:8080
if you want to start jenkins on a different port, update port number in 
10. /etc/sysconfig/jenkins
To fetch initial admin password, do:
11. sudo su - 
12. cd /var/lib/jenkins/secrets/
13. cat initialAdminPassword
to uninstall jenkins
14. sudo service jenkins stop
15. sudo yum remove jenkins
16. sudo rm -r /var/lib/jenkins #this removes jenkins folders

# How to install terraform in aws ec2
1. Get terraform package: wget https://releases.hashicorp.com/terraform/0.11.13/terraform_0.11.13_linux_amd64.zip
2. Unzip the package: unzip terraform_0.11.13_linux_amd64.zip
3. Move the package to /usr/local/bin: sudo mv terraform /usr/local/bin/ && rm terraform_0.11.13_linux_amd64.zip
4. check if terraform is working properly: terraform --version

To create a file to test the terraform
1. Create directory and get inside it: mkdir terraform && cd terraform
2. Create your 1st terraform script: vi main.tf
3. Create a VPC with cidr block 10.0.0.0/16:

In [ ]:
provider "aws" {
    access_key = "<your_access_key>"
    secret_key = "<your_secret_key>"
    region = "ap-south-1"
}

resource "aws_vpc" "myvpc" {
    cidr_block = "10.0.0.0/16"

    tags {
        name = "myvpc"
    }
}

Auto format the terraform files:
terraform fmt

Initialize terraform
terraform init

create the terraform plan
terraform plan

create the resources, type yes when prompted
terraform apply

Delete the resources, type yes when prompted
terraform destroy

check files (due to above destroy, terraform.tfstate fiel empties and terraform.tfstate.backup created as backup)
ls
cat terraform.tfstate
cat terraform.tfstate.backup

To cleanup
cd .. && rm -rf terraform

#To Build Terraform Pipeline in Jenkins
1. Install java
2. Install Jenkins
3. Install git
4. Install terraform = download terraform, then unzip it and do the above

After installing Jenkins:
1. go to manage jenkins
2. go to manage plugins
3. to to available
4. search of terraform
5. select terraform and click install without restart
6. go back to top of page

In order to configure terraform
1. go to manage jenkins
2. go to global tool configuration
3. scroll down to terraform, click add terraform
4. add name: terraform
5. chose a version that suites your OS, such as Linux amd64
6. better do it manually by unchecking the install automatically, then define a location, eg. terraform installed directory: /usr/bin
7. apply and save

Configure your variables
1. go to manage credentials
2. go to jenkins
3. go to global credentials
4. go to add credentials
5. go to kind: select from the drop down: secret text
6. add ID: github_api: CHECK BELOW ON HOW TO GET THAT
7. copy the secret token and paste on secret
8. click okay

#6 above: To get github_api
1. go to user icon, select setting
2. go to developer settings
3. go to personal access tokens
4. click on generate new token
5. add note: my-test-token
6. select scopes: admin:org, and delete_repo (be very careful on permission you give here)
7. click generate token
8. copy token to

Back to Jenkins
1. manage jenkins
2. configure system
3. scroll down to github
4. click on on Add GitHub server
5. add name: github_api
6. its api url: https://apigithub.com
7. select credential dropdown: github_api
8. check manage hooks and 
9. click on Test connection
10. apply and save

Now to Import few more credentials
1. manage jenkins
2. manage credentials
3. go to jenkins
4. go to global credentials
5. go to add credentials
6. select from kind dropdown: secret text
7. add ID: AWS_ACCESS_KEY
8. copy the secret value and paste
9. click ok
10. add credentials again: secret text, ID as AWS_SECRET_ACCESS_KEY, paste secret value
11. click 

Now create a Jenkins job
1. On the dashboard, click create a job
2. enter an item name: jenkins-terraform-pipeline
3. select pipeline on the list down
4. click ok
5. Now under Build Triggers, ensure you check: GitHub hook trigger for GITScm polling (TO BE DISCUSS BELOW)
6. under pipeline, select: Pipeline script from SCM
7. under SCM, select: Git
8. under repository url: copy git repo url: example: the public url of the file path
9. under script path: copy the jenkins file path: eg. jenkins-terraform-pipeline/ec2_pipeline, you have to add /jenkinsfile
10. apply and save

Return to dashboard
1. go to job: Pipeline jenkins-terraform-pipeline
2. click build now

To view the job:
1. click the job under build history

To see console output
1. click on console output

to see it stages:
1. return to status or dashboad.

Now to create automatic workflow, you will need to configure github webhooks
1. in the repository bar, go to settings
2. go to webhooks
3. go to add webhook
4. under payload url, copy your jenkins url and paste there: example, http:/34.222.195.13:8080/, then add github-webhook/ to it
5. under content type, select: application/json
6. under which events would you like to trigger this webhook?: select: send me everything
7. click add webhook

#GIT merge Permission
for a repos
1. go to repos bar, click on setting
2. go to branches
3. under branches protection rules, click on add rule
4. add Branch name pattern: master
5. under protection matching branches, check rules you want to appliy: Require pull request reviews before merging, Require status checks topass before merging, Require branches to be up to date before merging
6. click on create

From here now, other users will need to create a branch and create pull request. the pull request has to be reviewed and merged by the admin user. This is because of this branch protection

New Branch (Branch Protection) --> Pull Request --> Reviewing --> Merge --> Master